In [1]:
import pandas as pd                        ## Dataframe manipulation
import requests                          ##
from requests_html import HTMLSession  ## We dont have to manipulate the webpage at all, so a chrome based scraper is not needed
from bs4 import BeautifulSoup as bs  ## how we read the info from the request
import csv                         ## how we will store the data portably

In [2]:
def searchDF(df, column, query):
    searchDF = df[df[column].apply(str).str.contains(query, na=False)]
    return searchDF


In [3]:
def readBill(billUrl):
            ##   This currently only gets the most recent text of the bill.
            ##  This page has much more info, particularly the bill revisions
    htmlRequester = HTMLSession()
    r = htmlRequester.get('http://' + billUrl)
    soup = bs(r.text, 'html.parser')
    billcard = soup.find_all(class_="card-body")
    billtextUrl = 'http://revisor.mn.gov/bills/' + billcard[0].find('a')['href'].strip('/bills')
    r = htmlRequester.get(billtextUrl)
    soup = bs(r.text, 'html.parser')
    billText = soup.find(id='document').text.replace('\n', '')
    return billText

In [4]:
def resumeScrape(finalDF):
    urlsToCheck = len(searchDF(finalDF, 'scrapeComplete', 'False')['billUrl'].unique())
    for count, url in enumerate(searchDF(finalDF, 'scrapeComplete', 'False')['billUrl'].unique()):
        indexes = finalDF.loc[finalDF['billUrl'] == url].index
        indexes
        finalDF.at[indexes[0], 'text'] = readBill(url)
        finalDF.at[indexes[0], 'scrapeComplete'] = 'True'

        for index in indexes[1:]:
            finalDF.at[index, 'text'] = 'duplicate'
            finalDF.at[index, 'scrapeComplete'] = 'True'
            
        
        if count % 5 == 0:
            print(count, ' of ', urlsToCheck)
        
        if count >  5:     ##  DELETE THESE LINES TO UNLIMIT DEMO
            return finalDF  ##   CURRENTLY ONLY CHECKS 5 Bills a Session
                             ##   Limiting Legislature, Session Below as well (search limit)

                               ## Maybe should write line out to csv instead of holding in DF?
    return finalDF
    


In [5]:
fileName = 'mnLaws.csv'
try:
    dataframe = pd.read_csv(fileName)
    finalDF = resumeScrape(dataFrame)
    finalDF.to_csv(fileName)
except :
    with open(fileName, 'a', newline='') as csvfile:
        csvWriter = csv.writer(csvfile, delimiter=' ')
        header = ['LegislatureName', 'LegislatureUrl', 'sessionYear', 'sessionType',
       'sessionUrl', 'chapter', 'chapterUrl', 'bill', 'billUrl', 'text',
       'PresentmentDate', 'scrapeComplete']
        ## The DOM breaks it down into bill sections and subdivisions,
        ##but we just need the text for todays project
        csvWriter.writerow(header)
        print('new file created')


new file created


In [6]:
htmlRequester = HTMLSession()
# r = requests.get('https://www.revisor.mn.gov/laws/')  ## if just using requests, not requestsHTML, maybe we can test the speed
r = htmlRequester.get('https://www.revisor.mn.gov/laws/')  ## or efficency of these three methods (requests, html, chromium)
soup = bs(r.text, 'html.parser')

In [7]:
## Exploring the data a bit

In [8]:
soup.title

<title>
          
            MN Laws
          
        </title>

In [9]:
legislatures = soup.find_all('tr','alternate')  ## Using the inspect tool here to find the identifier for the data we want
legislatures[0]

<tr class="alternate">
<td>
<a href="//www.revisor.mn.gov/laws/91.0">
                    
                        2019 - 2020
                    
                    </a>
</td>
<td>
                    91st Legislature
                </td>
</tr>

In [10]:
legislatures[0].find('a', href=True)['href']  

'//www.revisor.mn.gov/laws/91.0'

In [11]:
legislatures[0].find_all('td')[1].text.strip('\n ').rstrip('\n ')

'91st Legislature'

In [12]:
sessionTracker = pd.DataFrame(data=None, index=None, columns= ['LegislatureName', 'LegislatureUrl'])
print(sessionTracker)


yearlysessionTracker = pd.DataFrame(data=None, index=None, columns= ['LegislatureName', 'sessionYear', 'sessionType', 'sessionUrl'])
print(yearlysessionTracker)

Empty DataFrame
Columns: [LegislatureName, LegislatureUrl]
Index: []
Empty DataFrame
Columns: [LegislatureName, sessionYear, sessionType, sessionUrl]
Index: []


In [13]:
## COLLECTING LIST OF LEGISLATURES  (I didnt turn these into functions because each page is too different)

for session in legislatures:
    name = session.find_all('td')[1].text.strip('\n ').rstrip('\n ')
    url = session.find('a', href=True)['href'].strip('//')
    sessionTracker.loc[len(sessionTracker.index)] = [name, url]
sessionTracker.head()
      ## TODO We are missing the 92nd legislature // Most recent posting, it has a different layout, manualy added 
newRow = pd.DataFrame({'LegislatureName':'92nd Legislature', 'LegislatureUrl':'www.revisor.mn.gov/laws/92.0'},index=[0])
sessionTracker = pd.concat([newRow, sessionTracker]).reset_index(drop = True)
sessionTracker.head()

,LegislatureName,LegislatureUrl
0,92nd Legislature,www.revisor.mn.gov/laws/92.0
1,91st Legislature,www.revisor.mn.gov/laws/91.0
2,89th Legislature,www.revisor.mn.gov/laws/89.0
3,87th Legislature,www.revisor.mn.gov/laws/87.0
4,85th Legislature,www.revisor.mn.gov/laws/85.0


In [14]:
# Collecting List of Sessions in a Legislature

iterator = 0        
while iterator < 5:  ## Limit 5
# while iterator < len(sessionTracker.index):
    sessionName = sessionTracker.loc[iterator]['LegislatureName']
    url = sessionTracker.loc[iterator]['LegislatureUrl']

    r = htmlRequester.get('http://' + url)
    soup = bs(r.text, 'html.parser')
    sessions = soup.find_all('p', 'p_session')
    for session in sessions:
        yearlySession = session.find_all('a')[0].text.replace('\n', '').strip(' ').rstrip(' ')

        sessionYear = yearlySession[0:4]
        sessionType = yearlySession[6:].strip(' ')
        sessionUrl = sessions[0].find_all('a', href=True)[0]['href'].strip('//')
        yearlysessionTracker.loc[len(yearlysessionTracker.index)] = [sessionName, sessionYear, sessionType, sessionUrl]
    iterator += 1

In [15]:
yearlysessionTracker.head(5)                         

,LegislatureName,sessionYear,sessionType,sessionUrl
0,92nd Legislature,2022,Regular Session,www.revisor.mn.gov/laws/2022/0
1,92nd Legislature,2021,Regular Session,www.revisor.mn.gov/laws/2022/0
2,92nd Legislature,2021,1st Special Session,www.revisor.mn.gov/laws/2022/0
3,91st Legislature,2020,Regular Session,www.revisor.mn.gov/laws/2020/0
4,91st Legislature,2020,1st Special Session,www.revisor.mn.gov/laws/2020/0


In [16]:
billTracker = pd.DataFrame(data=None, index=None, columns= (['sessionUrl', 'chapter',  'chapterUrl', 'bill', 'billUrl', 'text', 'PresentmentDate', 'scrapeComplete']))
billTracker.head()

,sessionUrl,chapter,chapterUrl,bill,billUrl,text,PresentmentDate,scrapeComplete


In [17]:
## Collecting List Bills in  a session
iterator = 0 
while iterator < 5: ## Limit 5
# while iterator < len(yearlysessionTracker.index):

    sessionUrl = yearlysessionTracker.loc[iterator]['sessionUrl']
    r = htmlRequester.get('http://' + sessionUrl)
    soup = bs(r.text, 'html.parser')
    chapters = soup.find_all('tr')
    try:
        for chapter in chapters[1:]:
            chapterUrl = chapter.find_all('a',href=True)[0]['href'].strip('//')
            chapterName =chapter.find_all('a',href=True)[0].text[7:]
            BillUrl = 'revisor.mn.gov/' + chapter.find_all('a',href=True)[1]['href'].strip('//')
            BillName = chapter.find_all('a',href=True)[1].text
            PresentmentDate = chapter.find_all('td')[2].text
            text = ""
            billTracker.loc[len(billTracker.index)] = [sessionUrl, chapterName, chapterUrl, BillName, BillUrl, text, PresentmentDate, False]
    except:
        billTracker.loc[len(billTracker.index)] = [sessionUrl, chapterName, chapterUrl, BillName, BillUrl, text, PresentmentDate, 'ERROR']
    iterator += 1
## TODO ASK DOMAIN EXPERT ABOUT CHAPTERS,for now we just continue towards bill text

In [18]:
billTracker.head()

,sessionUrl,chapter,chapterUrl,bill,billUrl,text,PresentmentDate,scrapeComplete
0,www.revisor.mn.gov/laws/2022/0,32,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,HF1203,revisor.mn.gov/bills/bill.php?b=house&f=hf1203...,,"February 3, 2022",False
1,www.revisor.mn.gov/laws/2022/0,33,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,SF2774,revisor.mn.gov/bills/bill.php?b=senate&f=sf277...,,"March 3, 2022",False
2,www.revisor.mn.gov/laws/2022/0,34,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,SF2957,revisor.mn.gov/bills/bill.php?b=senate&f=sf295...,,"March 3, 2022",False
3,www.revisor.mn.gov/laws/2022/0,35,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,SF1782,revisor.mn.gov/bills/bill.php?b=senate&f=sf178...,,"March 3, 2022",False
4,www.revisor.mn.gov/laws/2022/0,36,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,HF2454,revisor.mn.gov/bills/bill.php?b=house&f=hf2454...,,"March 10, 2022",False


In [19]:
resumeScrape(billTracker)  ## search 'limit' to find 3 limiters to run full scan

0  of  121
5  of  121


,sessionUrl,chapter,chapterUrl,bill,billUrl,text,PresentmentDate,scrapeComplete
0,www.revisor.mn.gov/laws/2022/0,32,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,HF1203,revisor.mn.gov/bills/bill.php?b=house&f=hf1203...,A bill for an act relating to labor and indus...,"February 3, 2022",True
1,www.revisor.mn.gov/laws/2022/0,33,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,SF2774,revisor.mn.gov/bills/bill.php?b=senate&f=sf277...,A bill for an act relating to human services;...,"March 3, 2022",True
2,www.revisor.mn.gov/laws/2022/0,34,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,SF2957,revisor.mn.gov/bills/bill.php?b=senate&f=sf295...,A bill for an act relating to retirement; Min...,"March 3, 2022",True
3,www.revisor.mn.gov/laws/2022/0,35,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,SF1782,revisor.mn.gov/bills/bill.php?b=senate&f=sf178...,A bill for an act relating to local governmen...,"March 3, 2022",True
4,www.revisor.mn.gov/laws/2022/0,36,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,HF2454,revisor.mn.gov/bills/bill.php?b=house&f=hf2454...,A bill for an act relating to local governmen...,"March 10, 2022",True
...,...,...,...,...,...,...,...,...
305,www.revisor.mn.gov/laws/2020/0,114,www.revisor.mn.gov/laws/2020/0/Session+Law/Cha...,SF3204,revisor.mn.gov/bills/bill.php?b=senate&f=sf320...,,"May 18, 2020",False
306,www.revisor.mn.gov/laws/2020/0,115,www.revisor.mn.gov/laws/2020/0/Session+Law/Cha...,SF13,revisor.mn.gov/bills/bill.php?b=senate&f=sf13&...,,"May 18, 2020",False
307,www.revisor.mn.gov/laws/2020/0,116,www.revisor.mn.gov/laws/2020/0/Session+Law/Cha...,HF4415,revisor.mn.gov/bills/bill.php?b=house&f=hf4415...,,"May 18, 2020",False
308,www.revisor.mn.gov/laws/2020/0,117,www.revisor.mn.gov/laws/2020/0/Session+Law/Cha...,SF3745,revisor.mn.gov/bills/bill.php?b=senate&f=sf374...,,"May 18, 2020",False


In [20]:
searchDF(billTracker, 'scrapeComplete', 'Error')


,sessionUrl,chapter,chapterUrl,bill,billUrl,text,PresentmentDate,scrapeComplete


In [21]:
## We made it down to the bill page, which has so much good info on it.  
## for today, we are just going to get the bill text

In [22]:
print(billTracker.head(1)['text'][0])
billTracker.head(1)


 A bill for an act relating to labor and industry; adopting recommendations of the 2022 Workers'  Compensation Advisory Council; extending the COVID-19 presumption for  workers' compensation eligibility; amending Minnesota Statutes 2020, sections  176.103, subdivision 3; 176.106, subdivision 7; 176.291; 176.295, subdivisions  1, 2; 176.305, subdivisions 1, 4; 176.321, subdivisions 2, 3; 176.331; 176.341,  subdivision 1; 176.391; 176.421, subdivision 4; Minnesota Statutes 2021  Supplement, sections 176.231, subdivision 9a; 176.2612, subdivision 3; Laws  2020, chapter 72, section 1, as amended; repealing Minnesota Statutes 2020, section  176.305, subdivision 2. BE IT ENACTED BY THE LEGISLATURE OF THE STATE OF MINNESOTA: ARTICLE 1 CLEANUP PROPOSALS Section 1. Minnesota Statutes 2020, section 176.103, subdivision 3, is amended to read: Subd. 3.Medical Services Review Board; selection; powers.(a) There is created a  Medical Services Review Board composed of the commissioner or the commissio

,sessionUrl,chapter,chapterUrl,bill,billUrl,text,PresentmentDate,scrapeComplete
0,www.revisor.mn.gov/laws/2022/0,32,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,HF1203,revisor.mn.gov/bills/bill.php?b=house&f=hf1203...,A bill for an act relating to labor and indus...,"February 3, 2022",True


In [23]:
sessionTracker.merge(yearlysessionTracker.merge(billTracker))

,LegislatureName,LegislatureUrl,sessionYear,sessionType,sessionUrl,chapter,chapterUrl,bill,billUrl,text,PresentmentDate,scrapeComplete
0,92nd Legislature,www.revisor.mn.gov/laws/92.0,2022,Regular Session,www.revisor.mn.gov/laws/2022/0,32,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,HF1203,revisor.mn.gov/bills/bill.php?b=house&f=hf1203...,A bill for an act relating to labor and indus...,"February 3, 2022",True
1,92nd Legislature,www.revisor.mn.gov/laws/92.0,2022,Regular Session,www.revisor.mn.gov/laws/2022/0,33,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,SF2774,revisor.mn.gov/bills/bill.php?b=senate&f=sf277...,A bill for an act relating to human services;...,"March 3, 2022",True
2,92nd Legislature,www.revisor.mn.gov/laws/92.0,2022,Regular Session,www.revisor.mn.gov/laws/2022/0,34,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,SF2957,revisor.mn.gov/bills/bill.php?b=senate&f=sf295...,A bill for an act relating to retirement; Min...,"March 3, 2022",True
3,92nd Legislature,www.revisor.mn.gov/laws/92.0,2022,Regular Session,www.revisor.mn.gov/laws/2022/0,35,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,SF1782,revisor.mn.gov/bills/bill.php?b=senate&f=sf178...,A bill for an act relating to local governmen...,"March 3, 2022",True
4,92nd Legislature,www.revisor.mn.gov/laws/92.0,2022,Regular Session,www.revisor.mn.gov/laws/2022/0,36,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,HF2454,revisor.mn.gov/bills/bill.php?b=house&f=hf2454...,A bill for an act relating to local governmen...,"March 10, 2022",True
...,...,...,...,...,...,...,...,...,...,...,...,...
1667,91st Legislature,www.revisor.mn.gov/laws/91.0,2019,1st Special Session,www.revisor.mn.gov/laws/2020/0,114,www.revisor.mn.gov/laws/2020/0/Session+Law/Cha...,SF3204,revisor.mn.gov/bills/bill.php?b=senate&f=sf320...,,"May 18, 2020",False
1668,91st Legislature,www.revisor.mn.gov/laws/91.0,2019,1st Special Session,www.revisor.mn.gov/laws/2020/0,115,www.revisor.mn.gov/laws/2020/0/Session+Law/Cha...,SF13,revisor.mn.gov/bills/bill.php?b=senate&f=sf13&...,,"May 18, 2020",False
1669,91st Legislature,www.revisor.mn.gov/laws/91.0,2019,1st Special Session,www.revisor.mn.gov/laws/2020/0,116,www.revisor.mn.gov/laws/2020/0/Session+Law/Cha...,HF4415,revisor.mn.gov/bills/bill.php?b=house&f=hf4415...,,"May 18, 2020",False
1670,91st Legislature,www.revisor.mn.gov/laws/91.0,2019,1st Special Session,www.revisor.mn.gov/laws/2020/0,117,www.revisor.mn.gov/laws/2020/0/Session+Law/Cha...,SF3745,revisor.mn.gov/bills/bill.php?b=senate&f=sf374...,,"May 18, 2020",False


In [24]:
sessionTracker.head(1)

,LegislatureName,LegislatureUrl
0,92nd Legislature,www.revisor.mn.gov/laws/92.0


In [25]:
yearlysessionTracker.head(1)

,LegislatureName,sessionYear,sessionType,sessionUrl
0,92nd Legislature,2022,Regular Session,www.revisor.mn.gov/laws/2022/0


In [26]:
billTracker.head(1)

,sessionUrl,chapter,chapterUrl,bill,billUrl,text,PresentmentDate,scrapeComplete
0,www.revisor.mn.gov/laws/2022/0,32,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,HF1203,revisor.mn.gov/bills/bill.php?b=house&f=hf1203...,A bill for an act relating to labor and indus...,"February 3, 2022",True


In [27]:
legislatureSessionCombo = sessionTracker.merge(yearlysessionTracker, how='outer')
legislatureSessionCombo.head(10)  

,LegislatureName,LegislatureUrl,sessionYear,sessionType,sessionUrl
0,92nd Legislature,www.revisor.mn.gov/laws/92.0,2022,Regular Session,www.revisor.mn.gov/laws/2022/0
1,92nd Legislature,www.revisor.mn.gov/laws/92.0,2021,Regular Session,www.revisor.mn.gov/laws/2022/0
2,92nd Legislature,www.revisor.mn.gov/laws/92.0,2021,1st Special Session,www.revisor.mn.gov/laws/2022/0
3,91st Legislature,www.revisor.mn.gov/laws/91.0,2020,Regular Session,www.revisor.mn.gov/laws/2020/0
4,91st Legislature,www.revisor.mn.gov/laws/91.0,2020,1st Special Session,www.revisor.mn.gov/laws/2020/0
5,91st Legislature,www.revisor.mn.gov/laws/91.0,2020,2nd Special Session,www.revisor.mn.gov/laws/2020/0
6,91st Legislature,www.revisor.mn.gov/laws/91.0,2020,3rd Special Session,www.revisor.mn.gov/laws/2020/0
7,91st Legislature,www.revisor.mn.gov/laws/91.0,2020,4th Special Session,www.revisor.mn.gov/laws/2020/0
8,91st Legislature,www.revisor.mn.gov/laws/91.0,2020,5th Special Session,www.revisor.mn.gov/laws/2020/0
9,91st Legislature,www.revisor.mn.gov/laws/91.0,2020,6th Special Session,www.revisor.mn.gov/laws/2020/0


In [28]:
legislatureSessionCombo.tail(10)  ## At this point i realized that the territoral legislatures were broken somehow
                                ## Examining the page showed a different style, fixing beyond scope of demo

,LegislatureName,LegislatureUrl,sessionYear,sessionType,sessionUrl
59,11th Legislature,www.revisor.mn.gov/laws/11.0,NaN,NaN,NaN
60,9th Legislature,www.revisor.mn.gov/laws/9.0,NaN,NaN,NaN
61,7th Legislature,www.revisor.mn.gov/laws/7.0,NaN,NaN,NaN
62,5th Legislature,www.revisor.mn.gov/laws/5.0,NaN,NaN,NaN
63,3rd Legislature,www.revisor.mn.gov/laws/3.0,NaN,NaN,NaN
64,1st Legislature,www.revisor.mn.gov/laws/1.0,NaN,NaN,NaN
65,7th Territorial Legislature,www.revisor.mn.gov/laws/0.7,NaN,NaN,NaN
66,5th Territorial Legislature,www.revisor.mn.gov/laws/0.5,NaN,NaN,NaN
67,3rd Territorial Legislature,www.revisor.mn.gov/laws/0.3,NaN,NaN,NaN
68,1st Territorial Legislature,www.revisor.mn.gov/laws/0.1,NaN,NaN,NaN


In [29]:
finalDF = legislatureSessionCombo.merge(billTracker, how='outer', on='sessionUrl')

In [30]:
finalDF.head(5)

,LegislatureName,LegislatureUrl,sessionYear,sessionType,sessionUrl,chapter,chapterUrl,bill,billUrl,text,PresentmentDate,scrapeComplete
0,92nd Legislature,www.revisor.mn.gov/laws/92.0,2022,Regular Session,www.revisor.mn.gov/laws/2022/0,32,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,HF1203,revisor.mn.gov/bills/bill.php?b=house&f=hf1203...,A bill for an act relating to labor and indus...,"February 3, 2022",True
1,92nd Legislature,www.revisor.mn.gov/laws/92.0,2022,Regular Session,www.revisor.mn.gov/laws/2022/0,33,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,SF2774,revisor.mn.gov/bills/bill.php?b=senate&f=sf277...,A bill for an act relating to human services;...,"March 3, 2022",True
2,92nd Legislature,www.revisor.mn.gov/laws/92.0,2022,Regular Session,www.revisor.mn.gov/laws/2022/0,34,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,SF2957,revisor.mn.gov/bills/bill.php?b=senate&f=sf295...,A bill for an act relating to retirement; Min...,"March 3, 2022",True
3,92nd Legislature,www.revisor.mn.gov/laws/92.0,2022,Regular Session,www.revisor.mn.gov/laws/2022/0,35,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,SF1782,revisor.mn.gov/bills/bill.php?b=senate&f=sf178...,A bill for an act relating to local governmen...,"March 3, 2022",True
4,92nd Legislature,www.revisor.mn.gov/laws/92.0,2022,Regular Session,www.revisor.mn.gov/laws/2022/0,36,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,HF2454,revisor.mn.gov/bills/bill.php?b=house&f=hf2454...,A bill for an act relating to local governmen...,"March 10, 2022",True


In [31]:
finalDF.to_csv('mnLaws.csv')

In [32]:
searchDF(finalDF, 'text', 'transportation')

,LegislatureName,LegislatureUrl,sessionYear,sessionType,sessionUrl,chapter,chapterUrl,bill,billUrl,text,PresentmentDate,scrapeComplete
1,92nd Legislature,www.revisor.mn.gov/laws/92.0,2022,Regular Session,www.revisor.mn.gov/laws/2022/0,33,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,SF2774,revisor.mn.gov/bills/bill.php?b=senate&f=sf277...,A bill for an act relating to human services;...,"March 3, 2022",True
6,92nd Legislature,www.revisor.mn.gov/laws/92.0,2022,Regular Session,www.revisor.mn.gov/laws/2022/0,38,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,HF3175,revisor.mn.gov/bills/bill.php?b=house&f=hf3175...,A bill for an act relating to local governmen...,"March 17, 2022",True
205,92nd Legislature,www.revisor.mn.gov/laws/92.0,2021,Regular Session,www.revisor.mn.gov/laws/2022/0,33,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,SF2774,revisor.mn.gov/bills/bill.php?b=senate&f=sf277...,A bill for an act relating to human services;...,"March 3, 2022",True
210,92nd Legislature,www.revisor.mn.gov/laws/92.0,2021,Regular Session,www.revisor.mn.gov/laws/2022/0,38,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,HF3175,revisor.mn.gov/bills/bill.php?b=house&f=hf3175...,A bill for an act relating to local governmen...,"March 17, 2022",True
409,92nd Legislature,www.revisor.mn.gov/laws/92.0,2021,1st Special Session,www.revisor.mn.gov/laws/2022/0,33,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,SF2774,revisor.mn.gov/bills/bill.php?b=senate&f=sf277...,A bill for an act relating to human services;...,"March 3, 2022",True
414,92nd Legislature,www.revisor.mn.gov/laws/92.0,2021,1st Special Session,www.revisor.mn.gov/laws/2022/0,38,www.revisor.mn.gov/laws/2022/0/Session+Law/Cha...,HF3175,revisor.mn.gov/bills/bill.php?b=house&f=hf3175...,A bill for an act relating to local governmen...,"March 17, 2022",True


In [33]:
searchDF(finalDF, 'text', 'eagle')  ## 92nd Legislature, 2021 1st Special Session
                                    ## from searchbar on revisor.gov
                                    ## couldnt find because query in CHAPTER text,
                                    ## not bill text,

,LegislatureName,LegislatureUrl,sessionYear,sessionType,sessionUrl,chapter,chapterUrl,bill,billUrl,text,PresentmentDate,scrapeComplete


In [34]:
# Thank you for reading! 